| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| C | array (Ny, Nx, 2, 2) | Cauchy-Green strain tensor over meshgrid |
| min_distance | float | minimum distance between local maxima |
| max_length | float | y-coordinate of vortex center |
| step_size | float | step size used for integration of shearline |
| incompressible | bool | Incompressible/Compressible |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add Algorithm folder to current working path
sys.path.append(parent_directory+"/General_Functions")

In [2]:
# Import function which calculates eigenvalues/eigenvectors in 2D
from ipynb.fs.defs.loc_max import _loc_max
from ipynb.fs.defs.orient_vector_field import _orient_vector_field
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines
from ipynb.fs.defs.eigen import eigen_S
import numpy as np

def _stretchlines(X, Y, S, min_distance, max_length, step_size, incompressible):
    
    s_1 = np.zeros((S.shape[0], S.shape[1]))*np.nan
    s_2 = np.zeros((S.shape[0], S.shape[1]))*np.nan
    v2 = np.zeros((S.shape[0], S.shape[1], 2))
    
    for i in range(s_1.shape[0]):
        for j in range(s_2.shape[1]):
            ls_1[i, j], s_2[i, j], _, v2[i,j,:] = eigen_S(S[i, j, :, :], incompressible)

    # Find local extrema of lambda_1-field
    peak_x, peak_y, peak_field = _loc_max(min_distance, X, Y, -s_1)
        
    # vector_field
    vector_field = v2
    
    # Defined domain
    defined_domain = np.isfinite(s_1).astype(int)
    
    for i in range(s_1.shape[0]):
        for j in range(s_1.shape[1]):
            if np.isnan(s_1[i, j]):
                s_1[i, j] = 0
                s_2[i, j] = 0
    
    # Import Rectangular bivariate spline from scipy
    from scipy.interpolate import RectBivariateSpline as RBS
        
    # Interpolate Eigenvalue field
    interp_s_1 = RBS(Y[:,0], X[0,:], np.nan_to_num(s_1), kx = 1, ky = 1)
    interp_s_2 = RBS(Y[:,0], X[0,:], np.nan_to_num(s_2), kx = 1, ky = 1)
    
    # Interpolate Eigenvector-field
    interp_vx = RBS(Y[:,0], X[0,:], np.nan_to_num(v2[:,:,0]), kx = 1, ky = 1)
    interp_vy = RBS(Y[:,0], X[0,:], np.nan_to_num(v2[:,:,1]), kx = 1, ky = 1)
    
    # Define list of stretchlines stretchlines  and associated s value
    stretchlines = [[], []]
    
    # Iterate over all local maxima
    for i in range(len(peak_x)):
        
        stretchlines_forw = [[], []]
        stretchlines_back = [[], []]
        
        # Local maxima point
        x = np.array([peak_x[i], peak_y[i]])
        
        # Boolean forward Iteration
        bool_forward, bool_backward = True, True
        
        # Check if local maximum is close to stretchlines 
        for j in range(len(stretchlines [0])):
                
            for k in range(len(stretchlines [0][j])):
                    
                if np.sqrt((x[0]-stretchlines[0][j][k])**2+(x[1]-stretchlines[1][j][k])**2) < min_distance:
                        bool_forward = False
                        bool_backward = False
                        break
        
        # Start integration only if local maxima is not close than 'min_distance' to stretchline
        if bool_forward and bool_backward:
            
            # Starting point of integration
            x_forward = x
            x_backward = x
            
            # Append starting point to list containing positions of forward stretchlines
            for ii in range(2):
                stretchlines_forw[ii].append(x[ii])
                
            # Check orientation of vector-field and rieorient if needed.
            vx, vy = _orient_vector_field(X, Y, x, vector_field)
            
            # Initial vector orientation
            x_prime_forward = np.array([vx, vy])
            x_prime_backward = -np.array([vx, vy])
            
            # Initial distance
            dist = 0
            
            # Start Integration with dummy variable 's_array'
            while bool_forward or bool_backward:
                
                # Integrate only if 'x_prime_forward' is defined and 'bool_forward == True'
                if bool_forward and x_prime_forward is not None:
                    
                    # RK4 integration for tensorline
                    x_forward, x_prime_forward = _RK4_tensorlines(X, Y, defined_domain, x_forward, x_prime_forward, step_size, vector_field, interp_s_2, interp_s_1) 
                        
                    if x_forward is not None:
                        
                        if np.sqrt(x_prime_forward[0]**2+x_prime_forward[1]**2) < 10**(-6):
                            bool_forward = False
                        
                        # Compute length of stretchline
                        dist += np.sqrt(x_prime_forward[0]**2+x_prime_forward[1]**2)*step_size
                        
                        # If distance is below length of stretchline --> append point to stretchline
                        if dist < max_length:
                            for ii in range(2):
                                stretchlines_forw[ii].append(x_forward[ii])
                    
                    else:
                        
                        bool_forward = False
                
                # Integrate only if 'x_prime_backward' is defined and 'bool_backward == True'
                if bool_backward and x_prime_backward is not None:
                    
                    # RK4 integration for tensorline
                    x_backward, x_prime_backward = _RK4_tensorlines(X, Y, defined_domain, x_backward, x_prime_backward, step_size, vector_field, interp_s_2, interp_s_1) 
                    
                    if x_backward is not None:
                        
                        if np.sqrt(x_prime_backward[0]**2+x_prime_backward[1]**2) < 10**(-6):
                            bool_backward = False
                        
                        # Compute stretchline
                        dist += np.sqrt(x_prime_backward[0]**2+x_prime_backward[1]**2)*step_size
                        
                        # If distance is below length of stretchline --> append point to stretchline
                        if dist < max_length:
                            for ii in range(2):
                                stretchlines_back[ii].append(x_backward[ii])
                    
                    else:
                        
                        bool_backward = False
                        
            # Append backward and forward stretchline
            for ii in range(2):
                stretchlines[ii].append(np.append(np.flip(stretchlines_back[ii]), stretchlines_forw[ii]))
    
    return stretchlines